In [ ]:
import librosa
import os
from tqdm import tqdm
import pandas as pd
import torch
import whisper

model = whisper.load_model("large")

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
def load_audio(audio_path):
    audio, sr = librosa.load(audio_path, sr=16000)
    return [audio, sr]

def get_audio_files(audio_dir):
    all_files = os.listdir(audio_dir)
    audio_files = [
        f for f in all_files
        if f.endswith('.opus') and not f.startswith('._')
    ]
    return audio_files

target_phrases = [
    "неслышу",
    "неслышно"
]

test_paths = get_audio_files("test_opus/audio")
answers = []
path_pred = []
with tqdm(test_paths, desc="Предсказание") as pbar:
    for path in pbar:
        file, sr = load_audio(f"{"test_opus/audio"}/{path}")
        result = model.transcribe(file, language='ru', fp16 = False)
        if any(phrase in result["text"].lower().replace(" ", "") for phrase in target_phrases):
            answers.append(1)
            a1 += 1
        else:
            answers.append(0)
            a0 += 1
        path_pred.append(path[0:-5])

submission_df = pd.DataFrame({'id': path_pred, 'label': answers})
submission_df.to_csv("answer412412412.csv", index=False)